In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [2]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import tqdm
from tqdm import tqdm
# !pip install downcast
from downcast import reduce

In [3]:
sales = pd.read_csv('sales_train_evaluation.csv')
# sales.name = 'sales'
calendar = pd.read_csv('calendar.csv')
# calendar.name = 'calendar'
prices = pd.read_csv('sell_prices.csv')
# prices.name = 'prices'

In [4]:
# replacing NaN value with 'No_event'
calendar.fillna('No_event',inplace=True)
calendar.head(5)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1


In [7]:
sales.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [ ]:
prices.head(5)

In [ ]:
calendar.head(5)

In [4]:
# replacing NaN value with 'No_event'
calendar.fillna('No_event',inplace=True)
calendar.head(5)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1


In [6]:
# replacing NaN value with 'No_event'
prices.fillna(0,inplace=True)

## 1. Time based feature 

### 1.1   Feature name: is_high_sale_months
* in EDA i found that February,march,april, may are the high sale month. so i am going to make one feature to represent high sale month

In [7]:
f=lambda x: 1 if x==2 or x==3 or x==4 or x==5 else 0
calendar['is_high_sale_months']=calendar['month'].map(f) 
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0,1
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,No_event,No_event,No_event,No_event,0,1,1,0
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,No_event,No_event,No_event,No_event,0,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,No_event,No_event,No_event,No_event,0,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,No_event,No_event,No_event,No_event,0,0,0,0


### 1.2 Feature name: is_high_sale_day
* in EDA i found that friday,saturday,sunday are the high sale day in every week.so i am giong to make feature to represent high sale day. 

In [8]:
f=lambda x: 1 if x==7 or x==1 or x==2 else 0
calendar['is_high_sale_day']=calendar['wday'].map(f) 
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0,0,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0,0,1
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,No_event,No_event,No_event,No_event,0,1,1,0,0
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,No_event,No_event,No_event,No_event,0,0,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,No_event,No_event,No_event,No_event,0,0,0,0,1
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,No_event,No_event,No_event,No_event,0,0,0,0,1


### 1.3 Feature name:is_christmas
* in EDA i found in every christmas sale is almost zero.so i am going to make feature to represent wheatehr day having christmas or not. 

In [9]:
f=lambda x: 1 if x=='Christmas' else 0
calendar['is_christmas']=calendar['event_name_1'].map(f)
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0,0,1,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0,0,1,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0,1,0,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,No_event,No_event,No_event,No_event,0,1,1,0,0,0
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,No_event,No_event,No_event,No_event,0,0,0,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,No_event,No_event,No_event,No_event,0,0,0,0,1,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,No_event,No_event,No_event,No_event,0,0,0,0,1,0


In [10]:
calendar[calendar['event_name_1']=='Christmas']

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas
330,2011-12-25,11148,Sunday,2,12,2011,d_331,Christmas,National,No_event,No_event,0,0,0,0,1,1
696,2012-12-25,11248,Tuesday,4,12,2012,d_697,Christmas,National,No_event,No_event,0,0,0,0,0,1
1061,2013-12-25,11348,Wednesday,5,12,2013,d_1062,Christmas,National,No_event,No_event,0,0,0,0,0,1
1426,2014-12-25,11447,Thursday,6,12,2014,d_1427,Christmas,National,No_event,No_event,0,0,0,0,0,1
1791,2015-12-25,11547,Friday,7,12,2015,d_1792,Christmas,National,No_event,No_event,0,0,0,0,1,1


In [11]:
df_sales=pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold')

In [ ]:
df_sales

In [16]:
df_final= pd.merge(df_sales, calendar, on='d', how='left')

MemoryError: Unable to allocate 452. MiB for an array with shape (59181090,) and data type int64

In [18]:
df_final= pd.merge(df_final, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

In [ ]:
df_final.info()

In [15]:
# Reducing memory size of DataFrame using Reduce function from downcasting library.
# https://medium.com/@deepakec1031/how-to-reduce-pandas-data-frame-size-using-downcast-e00cfed8e2c5
from downcast import reduce
def downcast(df):
     cols = df.dtypes.index.tolist()
     types = df.dtypes.values.tolist()
     for i,t in enumerate(types):
         if 'int' in str(t):
             if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                 df[cols[i]] = df[cols[i]].astype(np.int8)
             elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                 df[cols[i]] = df[cols[i]].astype(np.int16)
             elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                 df[cols[i]] = df[cols[i]].astype(np.int32)
             else:
                 df[cols[i]] = df[cols[i]].astype(np.int64)
         elif 'float' in str(t):
             if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                 df[cols[i]] = df[cols[i]].astype(np.float16)
             elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                 df[cols[i]] = df[cols[i]].astype(np.float32)
             else:
                 df[cols[i]] = df[cols[i]].astype(np.float64)
         elif t == np.dtype('object'):
             if cols[i] == 'date':
                 df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
             else:
                 df[cols[i]] = df[cols[i]].astype('category')
     return df  
#df_final= downcast(df_final)

In [17]:
df_sales = downcast(df_sales)


MemoryError: Unable to allocate 452. MiB for an array with shape (59181090,) and data type int64

In [ ]:
df_final.tail(5)

In [ ]:
df_final.info()

In [ ]:
df_final.head(5)

## 5. Label Encoding

In [9]:
import sklearn
from sklearn.preprocessing import LabelEncoder
Labelencoder=LabelEncoder() 
category=['event_name_1','event_type_1','event_name_2','event_type_2','id','item_id','dept_id','cat_id','store_id','state_id']
for i in tqdm(category):
    df_final[i+'_']=Labelencoder.fit_transform(df_final[i])

  0%|          | 0/10 [00:00<?, ?it/s]


NameError: name 'df_final' is not defined

In [ ]:
df_final=downcast(df_final)

In [ ]:
df_final.head(2)

In [ ]:
df_final=df_final.drop(['weekday','event_name_1','event_type_1','event_name_2','event_type_2','id','item_id','dept_id','cat_id','store_id','state_id'],axis = 1)

* converting day feature using numeric by removing hypen

In [ ]:
df_final

In [ ]:
l=[]
for i in df_final['d']:
    l.append(i.split('_')[1])
df_final['day']=l
df_final['day']=df_final['day'].astype(np.int16)

* droping some duplicate feature.

In [ ]:
df_final.head(-1)

In [ ]:
df_final=df_final.drop(['d'])

In [ ]:
df_final=downcast(df_final)

In [ ]:
df_final.head()

In [ ]:
LGBM=LGBMRegressor()

In [ ]:
id=df_final[['id_','day']]

In [ ]:
id

In [ ]:
x=df_final.drop(columns=['date','d','id_','sold'])

In [ ]:
y=df_final[['day','sold']]

In [ ]:
x = downcast(x)
y = downcast(y)

In [ ]:
x_train=x.loc[(x['day']>=916)&(x['day']<=1885)]
x_CV=x.loc[(x['day']>1885) &(x['day']<1914)]
x_test=x.loc[x['day']>=1914]

In [ ]:
print("x_train {}".format(x_train.shape))
print("x_CV {}".format(x_CV.shape))
print("x_test {}".format(x_test.shape))

In [ ]:
x_train.drop(columns=['day'],inplace=True)
x_CV.drop(columns=['day'],inplace=True)
x_test=x_test.drop(columns=['day'])

In [ ]:
y_train=y.loc[(x['day']>=916)&(y['day']<=1885)]
y_CV=y.loc[(y['day']>1885) &(y['day']<1914)]
y_test=y.loc[y['day']>=1914]

In [ ]:
print("y_train {}".format(y_train.shape))
print("y_CV {}".format(y_CV.shape))
print("y_test {}".format(y_test.shape))

In [ ]:
y_train.drop(columns=['day'],inplace=True)
y_CV.drop(columns=['day'],inplace=True)
y_test=y_test.drop(columns=['day'])

In [ ]:
y_train.info()
x_train.info()

# Modeling Starts.

## LGBM :

In [ ]:
x_train.head(2)

## MODEL SELECT And Tuning

In [18]:
"""def GridSearch(x_train,y_train,x_CV,y_CV,parameter):

  learning_rate=parameter['learning_rate']
  n_estimators=parameter['n_estimators']
  train_scores =[]
  CV_scores =[]
  parameter_str=[]
  for i in tqdm(n_estimators):
    for j in tqdm(learning_rate):
      clf=LGBMRegressor(objective='poisson' ,learning_rate=j,n_estimators=i)
      clf.fit(x_train,y_train)
      ## predicting CV data
      y_predict=clf.predict(x_CV)
      RMSE=mean_squared_error(y_CV, y_predict, squared=False)
      CV_scores.append(RMSE)
      ## predicting training data
      y_predict_=clf.predict(x_train)
      RMSE_=mean_squared_error(y_train, y_predict_, squared=False)
      train_scores.append(RMSE_)

      parameter_str.append('n_est:'+str(i)+'__'+'LR:'+str(j))
  return train_scores,CV_scores,parameter_str

parameters = {'learning_rate':[0.0001, 0.001, 0.01, 0.1,0.2,10] , 'n_estimators': [10,50,75,100,150] }
train_scores,CV_scores,parameter_str=GridSearch(x_train,y_train,x_CV,y_CV,parameters)"""

"def GridSearch(x_train,y_train,x_CV,y_CV,parameter):\n\n  learning_rate=parameter['learning_rate']\n  n_estimators=parameter['n_estimators']\n  train_scores =[]\n  CV_scores =[]\n  parameter_str=[]\n  for i in tqdm(n_estimators):\n    for j in tqdm(learning_rate):\n      clf=LGBMRegressor(objective='poisson' ,learning_rate=j,n_estimators=i)\n      clf.fit(x_train,y_train)\n      ## predicting CV data\n      y_predict=clf.predict(x_CV)\n      RMSE=mean_squared_error(y_CV, y_predict, squared=False)\n      CV_scores.append(RMSE)\n      ## predicting training data\n      y_predict_=clf.predict(x_train)\n      RMSE_=mean_squared_error(y_train, y_predict_, squared=False)\n      train_scores.append(RMSE_)\n\n      parameter_str.append('n_est:'+str(i)+'__'+'LR:'+str(j))\n  return train_scores,CV_scores,parameter_str\n\nparameters = {'learning_rate':[0.0001, 0.001, 0.01, 0.1,0.2,10] , 'n_estimators': [10,50,75,100,150] }\ntrain_scores,CV_scores,parameter_str=GridSearch(x_train,y_train,x_CV,y_C

In [ ]:
clf=LGBMRegressor(learning_rate=0.2,n_estimators=100)
clf.fit(x_train,y_train)
y_predict=clf.predict(x_test)
RMSE=mean_squared_error(y_test, y_predict, squared=False)
print('RMSE score on LGBM :',RMSE)

In [ ]:
x_train.info()

In [ ]:
imp=clf.feature_importances_
imp

In [ ]:
len(x_train.columns)

In [ ]:
features=['wday', 'month', 'snap_CA', 'snap_TX', 'snap_WI', 'is_high_sale_months',
       'is_high_sale_day', 'is_christmas', 'sell_prices','event_name_1_','event_type_1_', 'event_name_2_','event_type_2_', 'item_id_','dept_id_',
       'cat_id_', 'store_id_', 'state_id_','prices']

In [ ]:
len(features)

In [ ]:
data_=pd.DataFrame({'score':imp,'Features':features})

In [ ]:
data_=data_.sort_values(by='score',ignore_index=True)

In [ ]:
data_

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.bar(data_, x="score", y="Features", orientation='h')
fig.update_layout(
    autosize=False,
    width=700,
    height=700,
    )
fig.show()

### train our model on hyperparameter (with StandardScaler)

In [ ]:
clf=make_pipeline(StandardScaler(),LGBMRegressor(learning_rate=0.2,n_estimators=100))
clf.fit(x_train,y_train)
y_predict=clf.predict(x_test)
RMSE=mean_squared_error(y_test, y_predict, squared=False)
print('RMSE of linear regression with L2 regularizer :',RMSE)

## My task is to predict next 28 days sales for any particular product product.so i am writing two function the first function will take the product as input and return next 28 days for that product.(d_1914 to d_1942). the second function will take product and ground truth value as input and return RMSE score as output.

### Final model

In [4]:
def function_1(x):
  ## gathering  data into single dataframe  
  x=sales[sales['id']==x]
  df_sales=pd.melt(x, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold')

  # filling NaN values with no_event from calender features (event_name_1,event_type_1,event_name_2,event_type_ )
  calendar.fillna('No_event',inplace=True)
  df_final= pd.merge(df_sales, calendar, on='d', how='left')
  df_final= pd.merge(df_final, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

  ## filling NaN value with zero for sell_price column from df_final
  df_final.fillna(0,inplace=True)

  ## now its time to reduce memory size of df_final dataframe using downcasting technique 
  #df_final=reduce(df_final)


  ## now its time to do feature engineering 
  # 1. Time based feature
  # 1.1 Feature name: is_high_sale_months
  f=lambda x: 1 if x==2 or x==3 or x==4 or x==5 else 0
  df_final['is_high_sale_months']=df_final['month'].map(f) 

  # 1.2 Feature name: is_high_sale_day
  f=lambda x: 1 if x==7 or x==1 or x==2 else 0
  df_final['is_high_sale_day']=df_final['wday'].map(f) 

  # 1.3 Feature name:is_christmas
  f=lambda x: 1 if x=='Christmas' else 0
  df_final['is_christmas']=df_final['event_name_1'].map(f)


  ## 2. Label Encoding:in this we are going to convert categorical variable into numeric using sklearn preprocessing library called LabelEncoder
  import sklearn
  from sklearn.preprocessing import LabelEncoder
  labelencoder=LabelEncoder() 
  category=['event_name_1','event_type_1','event_name_2','event_type_2','item_id','dept_id','cat_id','store_id','state_id']
  for i in category:
      df_final[i+'_']=labelencoder.fit_transform(df_final[i])

  # droping duplicate fetures 
  df_final=df_final.drop(['event_name_1','event_type_1','event_name_2','event_type_2','item_id','dept_id','cat_id','store_id','state_id'],axis = 1)

  # removing hypen from "d" features : d_1-----> 1, d_100----->100
  l=[]
  for i in df_final['d']:
      l.append(i.split('_')[1])
  df_final['day']=l
  df_final['day']=df_final['day'].astype(np.int16)


  # dropping unusefull feature
  df_final=df_final.drop(['event_name_1_','event_name_2_','item_id_','d','date','wm_yr_wk','weekday','year'],axis = 1)
  


  ## storing id
  id=df_final[['id','day']]

  ## now dividing data into input and output form
  x=df_final.drop(columns=['id','sold'])
  y=df_final[['day','sold']]

  x_train=x.loc[(x['day']>=916)&(x['day']<=1885)]
  x_train.drop(columns=['day'],inplace=True)

  x_test=x.loc[x['day']>=1914]
  x_test=x_test.drop(columns=['day'])

  y_train=y.loc[(x['day']>=916)&(y['day']<=1885)]
  y_train.drop(columns=['day'],inplace=True)

  y_test=y.loc[y['day']>=1914]
  y_test=y_test.drop(columns=['day'])

  ## applying LGBM regressor
  clf=make_pipeline(StandardScaler(),LGBMRegressor(learning_rate=0.2,n_estimators=100))
  clf.fit(x_train,y_train)
  y_predict=clf.predict(x_test) 

  ## converting data into submission format 
  predicted_=pd.DataFrame({'sold_predicted':y_predict})
  predicted_=predicted_.apply(lambda x :round(x))
  predicted_= predicted_['sold_predicted'].astype('int16')
  
  id_index=id.reset_index()
  id_index.drop(columns='index',inplace=True)


  final_format=pd.merge(id_index, predicted_, left_index=True, right_index=True)
  final_format.pivot(index=['id'], columns='day')


  final_LGBM= final_format.pivot(index='id', columns='day')['sold_predicted'].reset_index()
  final_LGBM.columns.name = None


  return(final_LGBM,y_test,y_predict)

In [5]:
def function_2(y_test,y_predicted):
  ## calculating RMSE 
  RMSE=mean_squared_error(y_test['sold'], y_predicted, squared=True)
  print('RMSE using LGBMRegressor:',RMSE)

In [6]:
x=sales[sales['id']=='FOODS_3_384_CA_4_evaluation']
x

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
11755,FOODS_3_384_CA_4_evaluation,FOODS_3_384,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,1,1,1,1,0,0,0,0,0


In [10]:
import time
start=time.process_time()
final_LGBM,y_test,y_predicted=function_1('FOODS_3_384_CA_4_evaluation')
end=time.process_time()
total_time=end - start
print('total time required to predict next 28 days sale for one product is :',total_time)
final_LGBM

total time required to predict next 28 days sale for one product is : 2.65625


,id,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,FOODS_3_384_CA_4_evaluation,1,1,1,1,0,1,2,1,1,...,2,0,1,2,1,1,1,1,0,1


In [8]:
start=time.process_time()
function_2(y_test,y_predicted)
end=time.process_time()
total_time=end - start
print('total time required to RMSE score for one product is :',total_time)

RMSE using LGBMRegressor: 1.1774353229377987
total time required to RMSE score for one product is : 0.0
